In [12]:
import os
import pandas as pd
import instructor
import openai
from pydantic import Field, field_validator, BaseModel
from typing import Optional

In [8]:
res_df = pd.read_csv("/Users/shreeyadhakal/Desktop/Projects-2023/llama-3-finetune/infer_out/alpaca_cleaned_responses_nepali.csv")

In [9]:
res_df.head()

,instruction,input,output,id,generated_text,model_response,cleaned_response
0,"दिइएको क्यारेक्टर, सेटिङ र विषयवस्तु प्रयोग गर...","पात्रहरू: जासूस, वैज्ञानिक\nसेटिङ: परित्याग गर...",जासूस र वैज्ञानिकलाई एक रहस्यमय बेपत्ताको अनुस...,6916,<|begin_of_text|>Below is an instruction in Ne...,जासूस र वैज्ञानिक द्वारा कारखाना परित्याग गरिए...,जासूस र वैज्ञानिक द्वारा कारखाना परित्याग गरिए...
1,जलवायु परिवर्तनले मानिस र वातावरणमा पार्ने ५ प...,NaN,"1. आकस्मिक मृत्यु: गर्मी छाल, बाढी र अन्य जलवा...",36299,<|begin_of_text|>Below is an instruction in Ne...,1. जलवायु परिवर्तनको कारणहरू\n2. जलवायु परिवर्...,1. जलवायु परिवर्तनको कारणहरू\n2. जलवायु परिवर्...
2,उच्च प्रदर्शन इन्जिनलाई सही रूपमा वर्णन गर्ने ...,NaN,प्रदर्शनको पावरहाउस।,18066,<|begin_of_text|>Below is an instruction in Ne...,उच्च प्रदर्शन इन्जिनलाई सही रूपमा वर्णन गर्ने ...,उच्च प्रदर्शन इन्जिनलाई सही रूपमा वर्णन गर्ने ...
3,तपाईको विचारमा जलवायु परिवर्तनको प्रमुख कारण क...,NaN,जलवायु परिवर्तनको प्रमुख कारण जीवाश्म इन्धन जल...,3011,<|begin_of_text|>Below is an instruction in Ne...,तपाईको विचारमा जलवायु परिवर्तनको प्रमुख कारण क...,तपाईको विचारमा जलवायु परिवर्तनको प्रमुख कारण क...
4,गणितका ५ वटा शाखाको नाम दिनुहोस्।,NaN,- बीजगणित\n- ज्यामिति\n- क्याल्कुलस\n- संख्या ...,1520,<|begin_of_text|>Below is an instruction in Ne...,गणितका ५ वटा शाखा निम्नानुसार छन्:-\n१. अंकगणि...,गणितका ५ वटा शाखा निम्नानुसार छन्:-\n१. अंकगणि...


In [10]:
def load_api_key(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

In [51]:
class Score(BaseModel):
    relevance_score: Optional[int] = Field("Relevance score for response for a given instruction on a scale of 1 (lowest) to 10 (highest).")
    cc_score: Optional[int] = Field("Score how the response for clarity and coherence on a scale of 1 (lowest) to 10 (highest).")
    syntax_score: Optional[int] = Field("Score how the response for correctness of syntax and grammar on a scale of 1 (lowest) to 10 (highest).")
    complete_score: Optional[int] = Field("Score the response for completeness on a scale of 1 (lowest) to 10 (highest).")
    hallucination_type: Optional[str] = Field("Give the type of hallucination in the model response: factual, nonsensical or contradiction. If there is no hallucination say None.")
    is_repeat: Optional[str] = Field("Is the rensponse repeat of the Instruction or Input. Return Yes or No")


In [18]:
# class IsaRepeat(BaseModel):
    

#     @field_validator('is_repeat')
#     def score_validator(cls, s):
#         try:
#             if not isinstance(s, str):
#                 raise ValueError("The value must be an string.")
   
#             elif s.lower() not in ["yes", "no"]:
#                 raise ValueError("The response should be either yes or no.")
#             else:
#                 return s
#         except ValueError as e:
#             print(f"Validation error: {e}")

In [19]:
def get_content(input, ins, model_out):
    return f"""
        Grade the response of the model given the instruction and input.\n
        ### Start of Instruction and Input###

        {ins} + {input}

        ### End of Instruction ###
        ### Start of response ###

        {model_out}
        
        ### End of response ###
        """

In [40]:
from openai import OpenAI

client = instructor.from_openai(OpenAI())

In [62]:
from tqdm import tqdm 

scores = []

for idx, row in tqdm(res_df.iterrows(), total=len(res_df)):
    try:
        input = row["input"].strip()
    except Exception as e:
        input = ""
    ins = row ["instruction"].strip()
    expected = row["output"].strip()
    model_out = row["cleaned_response"].strip()
    res = {}
    if len(model_out) == 0:
        res["relevance_score"], res["cc_score"], res["syntax_score"], res["complete_score"] = 0, 0, 0, 0
        res["hallucination_type"] = "Empty Response"
        res["is_repeat"] = "Empty Response"

    else:
        try:
            response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                    {"role": "user", "content": get_content(input, ins, model_out)},
                ],
            response_model = Score,
            max_tokens=256,
            temperature=0.7,
            )
            res = response.model_dump()
            scores.append(res)
        except Exception as e:
            print(f"Falied to process row {idx} with exception {e}")

scores_df = pd.DataFrame(scores)

In [63]:
scores_df

,relevance_score,cc_score,syntax_score,complete_score,hallucination_type,is_repeat
0,2,2,2,1,nonsensical,No
1,1,2,4,2,nonsensical,No
2,1,1,1,1,None,Yes
3,1,1,1,1,None,Yes
4,5,5,5,5,factual,No
...,...,...,...,...,...,...
195,1,1,1,1,None,Yes
196,1,3,5,1,None,Yes
197,1,1,1,1,nonsensical,No
198,1,1,1,1,nonsensical,Yes


In [71]:
final_df = pd.concat([res_df, scores_df], axis=1)

In [72]:
final_df

,instruction,input,output,id,generated_text,model_response,cleaned_response,relevance_score,cc_score,syntax_score,complete_score,hallucination_type,is_repeat
0,"दिइएको क्यारेक्टर, सेटिङ र विषयवस्तु प्रयोग गर...","पात्रहरू: जासूस, वैज्ञानिक\nसेटिङ: परित्याग गर...",जासूस र वैज्ञानिकलाई एक रहस्यमय बेपत्ताको अनुस...,6916,<|begin_of_text|>Below is an instruction in Ne...,जासूस र वैज्ञानिक द्वारा कारखाना परित्याग गरिए...,जासूस र वैज्ञानिक द्वारा कारखाना परित्याग गरिए...,2,2,2,1,nonsensical,No
1,जलवायु परिवर्तनले मानिस र वातावरणमा पार्ने ५ प...,NaN,"1. आकस्मिक मृत्यु: गर्मी छाल, बाढी र अन्य जलवा...",36299,<|begin_of_text|>Below is an instruction in Ne...,1. जलवायु परिवर्तनको कारणहरू\n2. जलवायु परिवर्...,1. जलवायु परिवर्तनको कारणहरू\n2. जलवायु परिवर्...,1,2,4,2,nonsensical,No
2,उच्च प्रदर्शन इन्जिनलाई सही रूपमा वर्णन गर्ने ...,NaN,प्रदर्शनको पावरहाउस।,18066,<|begin_of_text|>Below is an instruction in Ne...,उच्च प्रदर्शन इन्जिनलाई सही रूपमा वर्णन गर्ने ...,उच्च प्रदर्शन इन्जिनलाई सही रूपमा वर्णन गर्ने ...,1,1,1,1,None,Yes
3,तपाईको विचारमा जलवायु परिवर्तनको प्रमुख कारण क...,NaN,जलवायु परिवर्तनको प्रमुख कारण जीवाश्म इन्धन जल...,3011,<|begin_of_text|>Below is an instruction in Ne...,तपाईको विचारमा जलवायु परिवर्तनको प्रमुख कारण क...,तपाईको विचारमा जलवायु परिवर्तनको प्रमुख कारण क...,1,1,1,1,None,Yes
4,गणितका ५ वटा शाखाको नाम दिनुहोस्।,NaN,- बीजगणित\n- ज्यामिति\n- क्याल्कुलस\n- संख्या ...,1520,<|begin_of_text|>Below is an instruction in Ne...,गणितका ५ वटा शाखा निम्नानुसार छन्:-\n१. अंकगणि...,गणितका ५ वटा शाखा निम्नानुसार छन्:-\n१. अंकगणि...,5,5,5,5,factual,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,एक समय को एक उदाहरण उत्पन्न गर्नुहोस् जब तपाईं...,NaN,मेरा सबै साथीहरूले पार्कमा ह्याङ्ग आउट गर्नको ...,16763,<|begin_of_text|>Below is an instruction in Ne...,एक समय को एक उदाहरण उत्पन्न गर्नुहोस् जब तपाईं...,एक समय को एक उदाहरण उत्पन्न गर्नुहोस् जब तपाईं...,1,1,1,1,None,Yes
196,यस लेखमा मुख्य विचारहरूको रूपरेखा सिर्जना गर्न...,डा. नोको कथा अङ्ग्रेजी लेखक इयान फ्लेमिङको छैट...,परिचय\nII। सेटिङ डा. नं\nए इंग्ल्याण्ड\nB. जमै...,32238,<|begin_of_text|>Below is an instruction in Ne...,यस लेखमा मुख्य विचारहरूको रूपरेखा सिर्जना गर्न...,यस लेखमा मुख्य विचारहरूको रूपरेखा सिर्जना गर्न...,1,3,5,1,None,Yes
197,निम्न पाठको टुक्रा लिनुहोस् र यसलाई थप औपचारिक...,नमस्ते! म हाम्रो नयाँ फोटोभोल्टिक विन्डो प्रणा...,हामी हाम्रो नयाँ फोटोभोल्टिक विन्डो प्रणालीको ...,35404,<|begin_of_text|>Below is an instruction in Ne...,"कलम, कागज, कापी र पेन, मेरो पन्जा मा हुन्।","कलम, कागज, कापी र पेन, मेरो पन्जा मा हुन्।<|en...",1,1,1,1,nonsensical,No
198,"कामको विवरण दिएर, महत्त्वपूर्ण योग्यताहरू सूची...",सफ्टवेयर इन्जिनियरको लागि जागिरको सूची।,सफ्टवेयर इन्जिनियरका लागि योग्यताहरूमा सफ्टवेय...,48622,<|begin_of_text|>Below is an instruction in Ne...,1. अनुभव\n2. ब्यबसायिक ज्ञान\n3. मूल्य\n4. सञ्...,1. अनुभव\n2. ब्यबसायिक ज्ञान\n3. मूल्य\n4. सञ्...,1,1,1,1,nonsensical,Yes


In [73]:
final_df.to_csv("../../infer_out/df_gpt_4_score_Nepali_baseline.csv")